
# Data Cleaning


Veri Seti: [Link](https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data/versions/2)  

In [ ]:
# paketleri import edelim
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# verileri pandas dataframe ine aktaralım...
df = pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['area_type'].unique()

In [ ]:
# gereksiz kolonları temizle
df = df.drop(['area_type','availability','society'],axis='columns')
df.head()

In [ ]:
df.shape

In [ ]:
# Null degerleri belirle ve gerekli işlemleri yap
df.isna().sum()

In [ ]:
df = df.dropna()
df.isna().sum()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df['size'].unique()

In [ ]:
int('3 BHK'.split(' ')[0])

In [ ]:
df['size']

In [ ]:
df['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df['oda'] = df['size'].apply(lambda x: int(x.split(' ')[0]))

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df = df.drop(['size'],axis='columns')
df.head()

In [ ]:
df.oda.unique()

In [ ]:
df[df.oda>20]

In [ ]:
df['total_sqft'].unique()

In [ ]:
def sayi_mi(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df[~df['total_sqft'].apply(sayi_mi)]

In [ ]:
df2 =  df[df['total_sqft'].apply(sayi_mi)==False]
df2.head()

In [ ]:
df2['total_sqft'].unique()

In [ ]:
df2[df2['total_sqft'].str.contains("Meter")]

In [ ]:
df.shape

In [ ]:
17/12710

In [ ]:
def convert_range_to_num(x):
    tokens = str(x).split('-')
    if len(tokens) == 2:
        res = (float(tokens[0]) + float(tokens[1])) / 2
    else:
        try:
            res = float(x)
        except:
            res = None
    return res

In [ ]:
x= '167Sq. Meter'
x= '12'
x= '10 - 20'
convert_range_to_num(x)

In [ ]:
convert_range_to_num('100 - 200')
# float('   1.32  ')

In [ ]:
df['total_sqft'] = df['total_sqft'].apply(convert_range_to_num)

In [ ]:
df['total_sqft'].isna().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

In [ ]:
df.total_sqft.unique()

In [ ]:
df.head()

# Feature Engineering

In [ ]:
df

![image](https://i.imgur.com/5hmuzns.png)

In [ ]:
df['total_sqft']

In [ ]:
df['metrekare'] = df['total_sqft'] * 0.092903

In [ ]:
df

In [ ]:
df['price_per_m2'] = df['price'] * 1_000 / df['metrekare']

In [ ]:
df

In [ ]:
len(df.location.unique())

In [ ]:
df.location.unique()

In [ ]:
df.location = df.location.apply(lambda x: x.strip())

In [ ]:
loc_stats = df.groupby(['location'])['location'].size().reset_index(name='counts').sort_values(by=['counts'],ascending=False)
loc_stats

In [ ]:
loc_stats

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

plt.xlabel("lokasyonlar counts değeri")
plt.ylabel("Kaç lokasyonda bu durumda")

bins = 10
data = loc_stats.counts
arr = plt.hist(data, bins=bins, range=(0,100))
for i in range(bins):
    plt.text(arr[1][i],arr[0][i],str(arr[0][i]))

In [ ]:
df.groupby(['location'])['location'].size().reset_index(name='counts').sort_values(by=['counts'],ascending=False)

In [ ]:
len(loc_stats[loc_stats.counts<=10])

In [ ]:
less_than_10 = loc_stats[loc_stats.counts<=10]
less_than_10

In [ ]:
#less_than_10.location.to_list()

In [ ]:
df.location =  df.location.apply(lambda x: 'other' if x in less_than_10.location.to_list() else x)

In [ ]:
df[df.location =='other']

In [ ]:
df.head(30)

In [ ]:
len(df.location.unique())

In [ ]:
236 / 1265

# Outlier Removal

In [ ]:
df

In [ ]:
600 * 0.092903

In [ ]:
55 / 6

In [ ]:
300 * 0.092903

In [ ]:
df[df['total_sqft']/df['oda']<300].head()

In [ ]:
df = df[~(df['total_sqft']/df['oda']<300)]
df.shape

In [ ]:
 df.price_per_m2.describe()

In [ ]:
def remove_ppm_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        lower_limit = subdf.price_per_m2.mean() - ( 1 * subdf.price_per_m2.std() )
        upper_limit = subdf.price_per_m2.mean() + ( 1 * subdf.price_per_m2.std() )
        
        reduced_df = subdf[(subdf.price_per_m2>lower_limit) & (subdf.price_per_m2<upper_limit)]
        df_out = pd.concat([df_out,reduced_df], ignore_index=True)
    
    return df_out

In [ ]:
df3 = remove_ppm_outliers(df)

In [ ]:
df.shape

In [ ]:
df3

![image](https://i.imgur.com/qROYwSi.png)

In [ ]:
def plot_scatter_chart(df,location):
    bhk2 = df[(df.location==location) & (df.oda==2)]
    bhk3 = df[(df.location==location) & (df.oda==3)]
    #matplotlib.rcParams['figure.figsize'] = (15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+', color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price (Lakh Indian Rupees)")
    plt.title(location)
    plt.legend()


In [ ]:
plot_scatter_chart(df3,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df3,"Hebbal")

**We should also remove properties where for same location, the price of (for example) 3 bedroom apartment is less than 2 bedroom apartment (with same square ft area). What we will do is for a given location, we will build a dictionary of stats per bhk, i.e.**
```
{
    '1' : {
        'mean': 4000,
        'std: 2000,
        'count': 34
    },
    '2' : {
        'mean': 4300,
        'std: 2300,
        'count': 22
    },    
}
```
**Now we can remove those 2 BHK apartments whose price_per_m2 is less than mean price_per_m2 of 1 BHK apartment**

In [ ]:
def remove_oda_outliers(df):
    exclude_indices = np.array([])
    for location, location_df in df.groupby('location'):
        oda_stats = {}
        for oda, oda_df in location_df.groupby('oda'):
            oda_stats[oda] = {
                'mean': np.mean(oda_df.price_per_m2),
                'std': np.std(oda_df.price_per_m2),
                'count': oda_df.shape[0]
            }
        for oda, oda_df in location_df.groupby('oda'):
            stats = oda_stats.get(oda-1)
            if stats and stats['count']>5:
                exclude_indices = np.append(exclude_indices, oda_df[oda_df.price_per_m2<(stats['mean'])].index.values)
    return df.drop(exclude_indices,axis='index')

In [ ]:
df3.shape

In [ ]:
df4 = remove_oda_outliers(df3)
df4.shape

In [ ]:
plot_scatter_chart(df4,"Rajaji Nagar")

In [ ]:
plot_scatter_chart(df4,"Hebbal")

In [ ]:
df4

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)
plt.hist(df4.price_per_m2,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df4.bath.unique()

In [ ]:
plt.hist(df4.bath,rwidth=0.8)
plt.xlabel("Number of bathrooms")
plt.ylabel("Count")

In [ ]:
df4[df4.bath>10]

In [ ]:
df4[df4.bath>df4.oda+2]

In [ ]:
df5 = df4[df4.bath<df4.oda+2]
df5.shape

In [ ]:
df5.head()

In [ ]:
df5.shape

In [ ]:
df6 = df5.drop(['balcony','price_per_m2','metrekare'],axis='columns')
df6.head(3)

In [ ]:
df6.rename(columns={'oda': 'bhk'},inplace=True, errors='raise')
df6.head(3)

In [ ]:
df6.shape

# Model Building

In [ ]:
df6.head()

In [ ]:
df6.location = df6.location.str.lower().str.strip()

In [ ]:
 pd.get_dummies(df6, drop_first=True, prefix='', prefix_sep='')

In [ ]:
df7 = pd.get_dummies(df6, drop_first=True, prefix='', prefix_sep='')
df7.head()

In [ ]:
df7.shape

In [ ]:
X = df7.drop(['price'],axis='columns')
X.head(3)

In [ ]:
y = df7.price
y.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)

In [ ]:
X_train.values

In [ ]:
y_train

In [ ]:
lr_clf = LinearRegression()
lr_clf.fit(X_train.values,y_train)
lr_clf.score(X_test.values,y_test)

<h2 style='color:blue'>Use K Fold cross validation to measure accuracy of our LinearRegression model</h2>

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

s = cross_val_score(LinearRegression(), X, y, cv=cv)
average = np.average(s)
average

**We can see that in 5 iterations we get a score above 80% all the time. This is pretty good but we want to test few other algorithms for regression to see if we can get even better score. We will use GridSearchCV for this purpose**

<h2 style='color:blue'>Find best model using GridSearchCV</h2>

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos = {
        'linear_regression' : {
            'model': LinearRegression(),
            'params': {}
        },
        'lasso': {
            'model': Lasso(),
            'params': {
                'alpha': [1,2],
                'selection': ['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params': {
                'criterion' : ["squared_error", "friedman_mse", "absolute_error","poisson"],
                'splitter': ['best','random']
            }
        }
    }
    scores = []
    cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
    for algo_name, config in algos.items():
        gs =  GridSearchCV(config['model'], config['params'], cv=cv, return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model': algo_name,
            'best_score': gs.best_score_,
            'best_params': gs.best_params_
        })

    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

find_best_model_using_gridsearchcv(X,y)

**Based on above results we can say that LinearRegression gives the best score. Hence we will use that.**

<h2 style='color:blue'>Test the model for few properties</h2>

In [ ]:
X.columns

In [ ]:
def predict_price(location,sqft,bath,bhk):
    location = str(location).lower().strip()
    loc_index = np.where(X.columns==location)[0][0]

    x = np.zeros(len(X.columns))
    x[0] = sqft
    x[1] = bath
    x[2] = bhk
    if loc_index >= 0:
        x[loc_index] = 1
        
    #print(x)

    return lr_clf.predict([x])[0]

In [ ]:
np.where(X.columns=='1st phase jp nagar')[0][0]

In [ ]:
np.zeros(len(X.columns))

In [ ]:
predict_price('1st Phase JP Nagar',1000, 2, 2)

In [ ]:
predict_price('1st Phase JP Nagar',1000, 3, 3)

In [ ]:
predict_price('Indira Nagar',1000, 2, 2)

In [ ]:
predict_price('Indira Nagar',1000, 3, 3)

<h2 style='color:blue'>Export the tested model to a pickle file</h2>

In [ ]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

<h2 style='color:blue'>Export location and column information to a file that will be useful later on in our prediction application</h2>

In [ ]:
import json
columns = {
    'data_columns' : [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))

In [ ]:
X.columns